In [2]:
import QuantLib as ql

# Time SubModule

The _ql/time_ sub-folder implements various time related classes. Lets take a look at the _Date_ object which can be constructed as _Date(date, month, year)_.

In [3]:
date = ql.Date(31, 3, 2015) # 31st March, 2015
print(date)

March 31st, 2015


In [4]:
date.dayOfMonth()

31

In [5]:
date.month()

3

In [6]:
date.year()

2015

In [10]:
date.weekday() == ql.Tuesday

True

In [20]:
# arithmetic with dates
date + 1 # add a day

Date(1,4,2015)

In [21]:
date - 1

Date(30,3,2015)

In [22]:
date + ql.Period(1, ql.Months)

Date(30,4,2015)

In [25]:
date + ql.Period(1, ql.Weeks)

Date(7,4,2015)

In [26]:
date + ql.Period(1, ql.Years)

Date(31,3,2016)

In [27]:
# logical operations
ql.Date(31, 3, 2015) > ql.Date(1, 3, 2015)

True

The _Schedule_ object can be used to construct a list of dates such as coupon payments.

In [29]:
date1 = ql.Date(1, 1, 2015)
date2 = ql.Date(1, 1, 2016)
tenor = ql.Period(ql.Monthly)
calender = ql.UnitedStates(ql.UnitedStates.NYSE)

In [32]:
schedule = ql.Schedule(date1, date2, tenor, calender, ql.Following, ql.Following, ql.DateGeneration.Forward, False)
list(schedule)

[Date(2,1,2015),
 Date(2,2,2015),
 Date(2,3,2015),
 Date(1,4,2015),
 Date(1,5,2015),
 Date(1,6,2015),
 Date(1,7,2015),
 Date(3,8,2015),
 Date(1,9,2015),
 Date(1,10,2015),
 Date(2,11,2015),
 Date(1,12,2015),
 Date(4,1,2016)]

Here we have generated a _Schedule_ object that will contain dates between _date1_ and _date2_ with the tenor specifying the _Period_ to be every Month. The _calendar_ object is used for determining holidays. The two arguments following the _calendar_ are the _BusinessDayConvention_. Here we chose the convention to be the day following holidays. That is why we see that holidays are excluded in the list of dates.

# Interest Rate

The _InterestRate_ class can be used to store the interest rate with the compounding type, day count and the frequency of compounding. Below we show how to create an interest rate of 5.0% compounded annually, using Actual/Actual day count convention.

In [35]:
annualRate = 0.05
dayCount = ql.ActualActual(ql.ActualActual.ISDA) # day count fraction for periods where the actual #days is considered
compoundType = ql.Compounded
frequency = ql.Annual

In [36]:
interestRate = ql.InterestRate(annualRate, dayCount, compoundType, frequency)

Lets say if you invest a dollar at the interest eate described by _interestRate_, the _compoundFactor_ method gives you how much your investment will be worth after _t_ years. Below we show that the value returned by _compoundFactor_ for 2 years agrees with the expected compounding formula.

In [37]:
interestRate.compoundFactor(2.0)

1.1025

In [45]:
first_annual_amt = 1 + (1*0.05*1) # P + PRT 
first_annual_amt 

1.05

In [46]:
second_annual_amt = first_annual_amt + (first_annual_amt*0.05*1)
second_annual_amt

1.1025

The _discountFactor_ method returns the reciprocal of the _compoundFactor_ method. The discount factor is useful while calculating the present value of future cashflows.

In [41]:
interestRate.discountFactor(2.0)

0.9070294784580498

In [42]:
1.0 / interestRate.compoundFactor(2.0)

0.9070294784580498

$0.907 is the present value or discount price of receiving $1.1025 in 2 years at a 5% interest rate.

A given interest rate can be converted into other types using the _equivalentRate_ 

In [47]:
newFrequency = ql.Semiannual
effectiveRate = interestRate.equivalentRate(compoundType, newFrequency, 1)
effectiveRate.rate()

0.04939015319191986

The _InterestRate_ class also has an _impliedRate_ method. The _impliedRate_ method takes compound factor to return the implied rate. The _impliedRate_ method is a static method in the _InterestRate_ class and can be used without an instance of _InterestRate_. Internally the _equivalentRate_ method invokes the _impliedRate_ method in its calculations. 

Here we have converted into a semi-annual compounding type. A 4.939% of semi-annual compounding is equivalent to 5.0% annual compounding. This should mean, that both should give identical discount factors.

In [50]:
interestRate.discountFactor(1.0)

0.9523809523809523

In [53]:
effectiveRate.discountFactor(1.0)

0.9523809523809521

So this means that pricing bonds using either interest rate convention should give the same net present value (barring some precision).

..means that when pricing bonds, the choice of interest rate convention (e.g., annual, semi-annual) should lead to the same net present value (NPV) of the bond,

# Conclusion
In this post we looked at the basics of QuantLib:

1. We learnt how to use _Date_ and _Schedule_ classes from the _time_ sub-module
2. We learnt how to use the _InterestRate_ class